In [1]:
import numpy as np
import scipy
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold  # If needed


In [2]:
PATH = os.getcwd()
print (PATH)

D:\capstone\Classification-of-Hyperspectral-Image-master


In [3]:
# Load the Global values (windowSize, numPCAcomponents, testRatio) from the text file global_variables.txt
myFile = open('global_variables.txt', 'r') 
file = myFile.readlines()[:]


for line in file:

    if line[0:3] == "win":

        ds = line.find('=')
        windowSize = int(line[ds+1:-1],10)

    elif line[0:3] == "num":

        ds = line.find('=')
        numPCAcomponents = int(line[ds+2:-1],10)

    else:

        ds = line.find('=')
        testRatio = float(line[ds+1:])


In [4]:
# Global Variables
#windowSize = 5
#numPCAcomponents = 30
#testRatio = 0.25

# Load Training Dataset

In [5]:
X_train = np.load("X_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio)  + ".npy")

y_train = np.load("y_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")

In [6]:
# Reshape into (numberofsamples, channels, height, width)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[3], X_train.shape[1], X_train.shape[2]))
y_train = to_categorical(y_train)

In [7]:
# Define the input shape 
input_shape= X_train[0].shape
print(input_shape)

(30, 5, 5)


In [8]:
# number of filters
C1 = 3*numPCAcomponents

In [9]:
# Define the model
model = Sequential()

model.add(Conv2D(C1, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(3*C1, (3, 3), activation='relu'))
model.add(Dropout(0.25))



model.add(Flatten())
model.add(Dense(6*numPCAcomponents, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

D:\capstone\Classification-of-Hyperspectral-Image-master\your_env_name\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
from tensorflow.keras.optimizers import SGD

# Update the SGD optimizer with correct parameters
sgd = SGD(learning_rate=0.0001, momentum=0.9, nesterov=True, weight_decay=1e-6)

# Compile your model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [11]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 28, 3, 90)           │           4,140 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 26, 1, 270)          │         218,970 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 26, 1, 270)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 7020)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 180)                 │       1,263,780 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 180)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             724 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,487,614 (5.67 MB)

 Trainable params: 1,487,614 (5.67 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
X_train.shape, y_train.shape

((15768, 30, 5, 5), (15768, 4))

In [13]:
model.fit(X_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 32s 62ms/step - accuracy: 0.7963 - loss: 0.5270
Epoch 2/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 41s 63ms/step - accuracy: 0.9147 - loss: 0.2245
Epoch 3/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 31s 63ms/step - accuracy: 0.9502 - loss: 0.1345
Epoch 4/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 28s 57ms/step - accuracy: 0.9708 - loss: 0.0825
Epoch 5/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 41s 56ms/step - accuracy: 0.9736 - loss: 0.0691
Epoch 6/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 27s 54ms/step - accuracy: 0.9842 - loss: 0.0480
Epoch 7/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 26s 53ms/step - accuracy: 0.9828 - loss: 0.0525
Epoch 8/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 28s 56ms/step - accuracy: 0.9869 - loss: 0.0395
Epoch 9/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 27s 55ms/step - accuracy: 0.9907 - loss: 0.0303
Epoch 10/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 40s 53ms/step - accuracy: 0.9872 - loss: 0.0395
Epoch 11/15
493/493 ━━━━━━━━━━━━━━━━━━━━ 27s 55ms/step - accuracy: 0.9919 - loss: 0.0212
Epoch 12/15
493/493 ━━━━━━━━━━

In [14]:
import h5py
from keras.models import load_model

In [15]:
model.save('my_model' + str(windowSize) + 'PCA' + str(numPCAcomponents) + "testRatio" + str(testRatio) + '.h5')


In [16]:
print(model.save)

<bound method Model.save of <Sequential name=sequential, built=True>>
